In [ ]:
import pandas as pd
import pdfplumber
from tqdm import tqdm
import os

In [ ]:
files_dir = '/Users/bp/Downloads/ds_documents'

In [ ]:
file_names = [f for f in os.listdir(files_dir) if f[:5] == 'PCSP_']
file_names 

In [ ]:
columns = ['service_code_and_description', 'mod', 'units', 'span', 'rate', 'start_date', 'end_date', 'self_directed', 'provider_responsible', 'status']
n_cols = len(columns)

In [ ]:
# for file_name in file_names:
# 6: random box
# 7: missing modifier
# 9: works but some extra on page2
# 12: only header on p1, rest on p2
# 15: works but some extra on page2
# 18: waiver services on p1, rest on p2
# bad = [6, 7]
# file_name = file_names[20]
# path = f'{files_dir}/{file_name}'
# opts = {"edge_min_length": 75}

In [ ]:
opts = {"edge_min_length": 75}

In [ ]:
def get_table_section_of_page(page):
    my_waiver_services = [d for d in page.extract_words(x_tolerance=3, y_tolerance=1, use_text_flow=True, keep_blank_chars=True)
                          if 'My Waiver Services' in d['text']][0]
    table_crop = (0, my_waiver_services['bottom'], page.width, page.height)
    return page.crop(table_crop )

In [ ]:
def get_name_dob_sid(page):
    name, dob, sid = page.extract_text().split('\n')[:3]
    name = name.replace('Member Name: ', '').strip()
    dob = dob.replace('DOB: ', '').strip()
    sid = sid.replace('SID: ', '').strip()
    return name, dob, sid

In [ ]:
def build_waver_services_df(page):
    lines = page.extract_table(opts)
    try:
        df = pd.DataFrame(lines[1:], columns=columns)
        original_rows = df.shape[0]
        df = df.loc[df.service_code_and_description != '']

        df['code'] = df.service_code_and_description.str.split('-').apply(lambda x: x[0])
        for col in df.columns:
            df[col] = df[col].str.replace('\n', ' ')
            
        return df
    
    except Exception as e:
        im = page.to_image()
        display(im.reset().debug_tablefinder(opts))
        display(e)
        display(lines)
        # raise

In [ ]:
def parse_waiver_services(path):
    pdf = pdfplumber.open(path)
    for i, page in enumerate(pdf.pages):
        page_text = page.extract_text().replace('\n', ' ')

        if 'Service Code and Description' in page_text or 'My Waiver Services' in page_text:
            break

    name, dob, sid = get_name_dob_sid(page)
    table_crop = get_table_section_of_page(page)
    # display(table_crop.to_image())
    df = build_waver_services_df(table_crop)
    
    next_page = pdf.pages[i+1]
    lines = next_page.extract_table()
    if df is not None and lines is not None and len(lines[0]) == df.shape[1]:
        if lines[0][0].strip() == '':
            for i, v in enumerate(lines[0]):
                df.iat[-1, i] = df.iat[-1, i] + v
        elif df is not None:
            df.append(build_waver_services_df(next_page))
        else:
            df = build_waver_services_df(next_page)
            
    if df is not None:
        df = df.assign(name=name, dob=dob, sid=sid)
    else:
        print(path)
    return df

In [ ]:
dfs = []
for file_name in tqdm(file_names):
    path = f'{files_dir}/{file_name}'
    df = parse_waiver_services(path)
    dfs.append(df)

combined_df = pd.concat(dfs)

combined_df.head()

In [ ]:
combined_df.to_csv('/Users/bp/Downloads/parsed_waivers.csv', index=False)

### workspace

In [ ]:
file_name = file_names[4]
path = f'{files_dir}/{file_name}'
opts = {"edge_min_length": 75}
pdf =  pdf = pdfplumber.open(path)

for i, page in tqdm(enumerate(pdf.pages)):
    if 'Service Code and Description' in page.extract_text() or 'My Waiver Services' in page.extract_text():
        break
        
im = page.to_image()
display(im.reset().debug_tablefinder(opts))

In [ ]:
# page = pdf.pages[i+1]
# im = page.to_image()
# im.reset().debug_tablefinder(opts)

In [ ]:
lines = page.extract_table()
df = pd.DataFrame(lines[3:], columns=columns)
df = df.loc[df.service_code_and_description != '']

df['code'] = df.service_code_and_description.str.split('-').apply(lambda x: x[0])
df['span'] = df.span.str.replace('\n', '')
df['start_date'] = df.start_date.str.replace('\n', '')
df['end_date'] = df.end_date.str.replace('\n', '')
df['provider_responsible'] = df.provider_responsible.str.replace('\n', ' ')
df = df[real_cols]
return df

In [ ]:
# im = page.to_image()
# im.reset().debug_tablefinder(opts)

In [ ]:
def build_df(lines):
    df = pd.DataFrame(lines[1:], columns=columns)
    original_rows = df.shape[0]
    df = df.loc[df.service_code_and_description != '']

    df['code'] = df.service_code_and_description.str.split('-').apply(lambda x: x[0])
    for col in df.columns:
        df[col] = df[col].str.replace('\n', ' ')
    # df = df[real_cols]
    return df
    
# df.head()

In [ ]:
def get_name_dob_sid(page):
    name, dob, sid = page.extract_text().split('\n')[:3]
    name = name.replace('Member Name: ', '').strip()
    dob = dob.replace('DOB: ', '').strip()
    sid = sid.replace('SID: ', '').strip()
    return name, dob, sid

In [ ]:
# if original_rows == df.shape[0]:
# if 'SCL and Habilitation information' not in next_page.extract_text():
next_page = pdf.pages[i+1]
lines = next_page.extract_table()
im = next_page.to_image()
im.reset().debug_tablefinder(opts)
# else:
#     # remove 
#     next_page = pdf.pages[i+1]
#     lines = next_page.extract_table()
#     im = next_page.to_image()
#     im.reset().debug_tablefinder()
#     print('no second page')
        

In [ ]:
# next_page = pdf.pages[i+1]
# im = next_page.to_image()
# im.reset().debug_tablefinder()

In [ ]:
if 'SCL and Habilitation information' not in next_page.extract_text():
    lines = next_page.extract_table()

In [ ]:
lines

In [ ]:
lines = page.extract_table()
if n_cols == len(lines[0]):